In [1]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

"""
DESCRIPTION:
    This sample demonstrates how to use agents with AI search.

USAGE:
    Before running the sample:

    pip install azure-ai-projects azure-ai-agents azure-identity

    Set these environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.

    Create an Azure AI search index and upload documents to it.
    Run 1- RAG - Azure AI Search - create embeddings.ipynb to create the index and upload documents.
    
"""

import os
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import CodeInterpreterTool, AzureAISearchTool
from azure.ai.agents.models import FilePurpose, MessageRole
from azure.identity import DefaultAzureCredential
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")
AIFOUNDRY_AISEARCH_CONNECTION_ID = os.getenv("AIFOUNDRY_AISEARCH_CONNECTION_ID")


In [2]:
from azure.ai.agents.models import AzureAISearchQueryType

project_client = AIProjectClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(),
)

agents_client = project_client.agents

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=AIFOUNDRY_AISEARCH_CONNECTION_ID, index_name="books", query_type=AzureAISearchQueryType.SIMPLE, top_k=3, filter=""
)

# Create agent with code interpreter tool and tools_resources
agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="my-aia-agent",
    instructions="You are helpful agent",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)
# [END upload_file_and_create_agent_with_code_interpreter]
print(f"Created agent, agent ID: {agent.id}")

thread = agents_client.threads.create()
print(f"Created thread, thread ID: {thread.id}")


Created agent, agent ID: asst_gGEf4bZFtBI6vnNvHJKUjhSw
Created thread, thread ID: thread_Rzu9PdoUAvgsfCEYQrD68sOE


In [ ]:
# Create a message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?",
)
print(f"Created message, message ID: {message.id}")

run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

Created message, message ID: msg_3FLOHMmdqoCoD5saXNDBhaXY
Run finished with status: completed


In [4]:
from azure.ai.agents.models import ListSortOrder

# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.role == MessageRole.AGENT and message.url_citation_annotations:
        placeholder_annotations = {
            annotation.text: f" [see {annotation.url_citation.title}] ({annotation.url_citation.url})"
            for annotation in message.url_citation_annotations
        }
        for message_text in message.text_messages:
            message_str = message_text.text.value
            for k, v in placeholder_annotations.items():
                message_str = message_str.replace(k, v)
            print(f"{message.role}: {message_str}")
    else:
        for message_text in message.text_messages:
            print(f"{message.role}: {message_text.text.value}")

user: Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?
assistant: Great question! The transformation of Queequeg’s coffin into Ishmael’s life buoy in *Moby-Dick* is both a key plot point and a powerful symbol.

### Plot Reason
After Queequeg recovers from a near-fatal illness, he decides to have a coffin made, believing he is about to die. When he unexpectedly regains his health, he no longer needs the coffin. Later, when the ship’s life buoys (lifebuoys) are found to be rotten and unusable, the carpenter repurposes Queequeg’s unused coffin into a life buoy by caulking it and adding a line so it floats.

When the Pequod is destroyed by Moby Dick, Ishmael alone survives the wreck. He clings to this very coffin-turned-life buoy until he is rescued:

> "I now know what had become of the boat. It had gone down at the whirl; and I was left afloat on the soft and gentle side... I floated on the coffin."

### Symbolic Meaning
The transformation carri